# find tracerRNA


In [1]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"




In [3]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [8]:
seqid ="FBDDNBEO_00645"
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  FBDDNBEO_00645
NODE_2092_length_11346_cov_6.0000_ID_47560
ZeeviD_2015
ZeeviD_2015__PNP_Main_471__bin.13
8770
1034
[('.', '27', '260')]
----------------------------------------------------------------------


## check there is only 1 Cas9 in that genome..

In [9]:
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
cas_dataset[cas_dataset["Genome Name"]==genomename]

,Seq ID,Seq Description,Seq,Contig,Genome Name,Study,Sample Name,SGB ID,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,uSGB,Level of estimated taxonomy,Estimated taxonomy
30968,FBDDNBEO_00645,FBDDNBEO_00645 CRISPR-associated endonuclease ...,MGAEFRHVTSLLLDEDFGEIKTAAGLRRQMRTRKAHKAREKWLERC...,NODE_2092_length_11346_cov_6.0000_ID_47560,ZeeviD_2015__PNP_Main_471__bin.13,ZeeviD_2015,PNP_Main_471,8770,1663 4767,"[('ID=CRISPR1', '.', '4')]","[('.', '27', '260')]","[('ID=FBDDNBEO_00644', '776', '1666')]","[('ID=FBDDNBEO_00643', '378', '728')]",Yes,Other,-


In [13]:
cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

print("..working genomename:", filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset))
########################################################################

# get CRISPR spacer and repeat sequence

cr=locus.CRISPRarray(feature=feature, contigname=contigname, genomename=genomename, datasetname=dataset)
cr.get_CRISPR_array()
spacers=cr.spacers
repeats=cr.repeats
repeat_start_pos=cr.repstartpos
print("cp "+cr.path+" "+datadir)


# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("+"*80)
print(cas_position)
print("Retrieving locus")
print("Copying file ot output...")
print("cp "+genomefilename+" "+datadir)

# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, datadir+"tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        temp_alt_cas9start=98503
        temp_alt_cas9stop=101700
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))
        
        #save .faa aminoacid sequence to output:
        if my_translated_cas9==cas9_aa:
            cas9aafilename=str(SGB)+"__"+seqid+"__"+genomename+"."+feature+".faa"
            f=open(datadir+cas9aafilename,"w") #TODO dir
            f.write(">"+seqid+"\n"+cas9_aa)
            f.close()

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        os.chdir(datadir)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        unique_repeats=np.unique(repeats)
        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db

        #3. make db file
        print("Building blast database with:")
        dbfile=datadir+"tempblastdb"
        print("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")
        os.system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")


        print("Running blastn of query file agianst all contigs")
        blastoutfile=datadir\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"
        print(blastn_command)
        os.system(blastn_command)
       # os.system("rm /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracerRNA/temp*")



-----------------------------------------------------
Cas9 id:	FBDDNBEO_00645
SGB:	8770
Genome Name:	 ZeeviD_2015__PNP_Main_471__bin.13
Contig:	NODE_2092_length_11346_cov_6.0000_ID_47560 
-----------------------------------------------------

..working genomename: ('ZeeviD_2015_B__PNP_Main_471_megahit__bin.13', 'ZeeviD_2015_B')


CRISPR 1   Range: 27 - 260

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

27		GTTTTGACTTTACAACAAAATGTTCTTTCTAAAAAT	GAAGAAAGCCGCTACGTCGCAGTCAGAATT	[ 36, 30 ]

93		GTTTTGACTTTACAACAAAATGTTCTTTCTAAAAAT	TTATAATTATAATAGAAAGGCACTAACTAT	[ 36, 30 ]

159		GTTTTGACTTTACAACAAAATGTTCTTTCTAAAAAT	CTATTTAGACCGCCTTGTTTAATTACAGTA	[ 36, 30 ]

225		GTTTTGACTTTACAACAAAATGTTCTTTCTAAAAAT	

--------	------------------------------------	------------------------------

Repeats: 4	Average Length: 36		Average Length: 30

cp /shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/1crisprsearch/out/ZeeviD_2015/ZeeviD_2015__PN

In [14]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['27', '93', '159', '225'] :
-----------------------------------------------------

rpt1|NODE_2092_length_11346_cov_6.0000_ID_47560|ZeeviD_2015__PNP_Main_471__bin.13|FBDDNBEO_00645	NODE_2092_length_11346_cov_6.0000_ID_47560	90.91	36	22	1	1	GACTTTACAACAAAA-TGTTCT	GACTTTACAACAAAAATTTTCT	318	297	6e-04	minus



## extract sequences

In [15]:
# manual input
matched_rep_start=297
matched_rep_stop=318

In [16]:
# this is the sequence matching the repeat:
matched_rep=Seq("GACTTTACAACAAAAATTTTCT")

#rpts which match:
rpt=unique_repeats[0]

# since repeat is on the + strand, and this is all on the + strand, tracrRNA should be its reverse complement:
antirepeat=matched_rep.reverse_complement()
#this should go frombo to bo  = x nt  (x+1)
print(antirepeat)
len(antirepeat)

AGAAAATTTTTGTTGTAAAGTC


22

In [18]:
# print terminators

f=open(datadir+str(SGB)+"__"+seqid+"__"+genomename+".terminators")
for line in f.readlines():
    if line.strip().startswith("3"): #manual input
        print(line)
f.close()

    319      Both + TTAACTGATTTTGCCGAATTTTAAGCTCTGCATGGCACCTTGGAATTCGGCATTTTTTTTCAAAA -8.60

   3264  Rnamotif + ATACGACTTTGTCGGGAACCCCGAaTTTTGATTCGAG -6.30

   3848      Both + AAATCGCCCTCGCCCCTTGCTTCCGAAAGGGGCcTTATCTTGCAAA -13.40



In [27]:
# if terminator on - strand
terminator_end=14560 -1  #manual input
terminator=Seq("AGATAATAAAAAATGTCCCGAGTGGGACATCTTTTATTTCTGGC".upper()) #manual input
print("terminator length: ", len(terminator))
terminator_start=terminator_end+len(terminator) #occhio che qui start e stop si riferisce al rev.compl, mentre nel antirepeat si riferisce alla direzione base

terminator length:  44


In [24]:
# if terminator on + strand
terminator_start=14560-1 #manual input
terminator=Seq("AGATAATAAAAAATGTCCCGAGTGGGACATCTTTTATTTCTGGC".upper()) #manual input
print("terminator length: ", len(terminator))
terminator_end=terminator_start+len(terminator)

terminator length:  44


In [23]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:
for record in SeqIO.parse(genomefilename, "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
print("test repeat: ", antirepeat, len(antirepeat), contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb
print(contigseq[matched_rep_start-1:matched_rep_stop-2].reverse_complement())
# check terminator
print("test terminator: " ,terminator, len(terminator), contigseq[terminator_end:terminator_start].reverse_complement()==terminator)


pre_tracrRNA=contigseq[terminator_end:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario=contigseq[terminator_start:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test repeat:  TTATAGGTTGCTTCTCAAATCGTCTTTGGTATAAT 35 False

test terminator:  AGATAATAAAAAATGTCCCGAGTGGGACATCTTTTATTTCTGGC 44 False
pre tracrRNA:  0
mario sequence:  0


ora ha una lunghezza simile, e finisce in modo simile a quello di _S. pyogenes_  TTTTTTTaltre

In [28]:
print(terminator_end, matched_rep_stop)

14559 14283


In [47]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch

alignments = pairwise2.align.localxx(rpt3, antirepeat.reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt3))


ATTGTAGTT--CCCTAATTT-TTCTTGGTATGTTATAAT
 |||||| |  ||| || || ||||||||||| | | ||
-TTGTAG-TAACCC-AA-TTGTTCTTGGTATG-T-T-AT
  Score=29

rpt length: 36


In [51]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx( rpt3,(antirepeat+mario[:4]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

A-TT---GTAGTT--CCCTAATTT-TTCTTGGTATGTTATAAT
  ||   |||| |  ||| || || ||||||||||| | | ||
-CTTTTTGTAG-TAACCC-AA-TTGTTCTTGGTATG-T-T-AT
  Score=29



In [ ]:
#no non fa nulla quel CTTT in più

In [54]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

print("\n----------------------------------------------------------------------")
print("         tracrRNA sequence for", feature,"protein id", seqid)
print("----------------------------------------------------------------------\n")
print("- contig name:", cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print("- Dataset: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print("- Bin: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print("- SGB: ",cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print("----------------------------------------------------------------------\n")
print("- Length: ", len(tracrRNA),"\n- position: ", terminator_end,matched_rep_stop)
print(tracrRNA)


----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id CJFLGLGO_01367
----------------------------------------------------------------------

- contig name: NODE_137_length_63930_cov_6.02239
- Dataset:  Obregon-TitoAJ_2015
- Bin:  Obregon-TitoAJ_2015__SM40__bin.4
- SGB:  4329
----------------------------------------------------------------------

- Length:  91 
- position:  49289 49380
TTATAACATACCAAGAACAATTGGGTTACTACAAAAAGGTAGAAAACCGAAAAGCTCTGACGGCTCCTTTTTGGAGCCGTTATCTTTTTCT
